## Imports + Commonality

## Multiplier Detection + Number Extractor

## Convert to Docling Common DataType

## Visualizer (Semantic En)

In [16]:
"""
Document structure visualization for Docling processed documents.
"""
from typing import List, Dict, Any, Optional
import pandas as pd
from docling_core.types.doc import DoclingDocument


class DocumentStructureVisualizer:
    """Extracts and organizes document structure from Docling documents."""

    def __init__(self, docling_document: DoclingDocument):
        """
        Initialize with a Docling document.

        Args:
            docling_document: The DoclingDocument object from conversion
        """
        self.doc = docling_document

    def get_document_hierarchy(self) -> List[Dict[str, Any]]:
        """
        Extract document hierarchy (headings and structure).

        Returns:
            List of dictionaries containing hierarchical structure info
        """
        hierarchy = []

        if not hasattr(self.doc, 'texts') or not self.doc.texts:
            return hierarchy

        for item in self.doc.texts:
            # Get label to identify item type
            label = getattr(item, 'label', None)
            label_str = str(label).lower() if label else ""

            # Focus on headers and titles
            if 'header' in label_str or 'title' in label_str:
                text = getattr(item, 'text', '')
                prov = getattr(item, 'prov', [])
                page_no = prov[0].page_no if prov else None

                hierarchy.append({
                    'type': label,
                    'text': text,
                    'page': page_no,
                    'level': self._infer_heading_level(label)
                })

        return hierarchy

    def _infer_heading_level(self, label: str) -> int:
        """Infer heading level from label."""
        if 'title' in label.lower():
            return 1
        elif 'section' in label.lower():
            return 2
        elif 'subsection' in label.lower():
            return 3
        else:
            return 4

    def get_tables_info(self) -> List[Dict[str, Any]]:
        """
        Extract table information and convert to DataFrames.

        Returns:
            List of dictionaries with table metadata and DataFrame
        """
        tables_info = []

        if not hasattr(self.doc, 'tables') or not self.doc.tables:
            return tables_info

        for i, table in enumerate(self.doc.tables, 1):
            try:
                # Export table to DataFrame
                df = table.export_to_dataframe(doc=self.doc)

                # Get provenance
                prov = getattr(table, 'prov', [])
                page_no = prov[0].page_no if prov else None

                # Get caption if available
                caption_text = getattr(table, 'caption_text', None)
                caption = caption_text if caption_text and not callable(caption_text) else None

                tables_info.append({
                    'table_number': i,
                    'page': page_no,
                    'caption': caption,
                    'dataframe': df,
                    'shape': df.shape,
                    'is_empty': df.empty
                })

            except Exception as e:
                # Handle tables that can't be converted
                print(f"Warning: Could not process table {i}: {e}")
                continue

        return tables_info

    def get_pictures_info(self) -> List[Dict[str, Any]]:
        """
        Extract picture/image metadata and image data.

        Returns:
            List of dictionaries with picture information and PIL images
        """
        pictures_info = []

        if not hasattr(self.doc, 'pictures') or not self.doc.pictures:
            return pictures_info

        for i, pic in enumerate(self.doc.pictures, 1):
            prov = getattr(pic, 'prov', [])

            if prov:
                page_no = prov[0].page_no
                bbox = prov[0].bbox

                # Get caption if available
                caption_text = getattr(pic, 'caption_text', None)
                caption = caption_text if caption_text and not callable(caption_text) else None

                # Get PIL image if available
                pil_image = None
                try:
                    if hasattr(pic, 'image') and pic.image is not None:
                        if hasattr(pic.image, 'pil_image'):
                            pil_image = pic.image.pil_image
                except Exception as e:
                    print(f"Warning: Could not extract image {i}: {e}")

                pictures_info.append({
                    'picture_number': i,
                    'page': page_no,
                    'caption': caption,
                    'pil_image': pil_image,  # Add PIL image
                    'bounding_box': {
                        'left': bbox.l,
                        'top': bbox.t,
                        'right': bbox.r,
                        'bottom': bbox.b
                    } if bbox else None
                })

        return pictures_info

    def get_document_summary(self) -> Dict[str, Any]:
        """
        Get overall document summary statistics.

        Returns:
            Dictionary with document statistics
        """
        pages = getattr(self.doc, 'pages', {})
        texts = getattr(self.doc, 'texts', [])
        tables = getattr(self.doc, 'tables', [])
        pictures = getattr(self.doc, 'pictures', [])

        # Count different text types
        text_types = {}
        for item in texts:
            label = getattr(item, 'label', 'unknown')
            text_types[label] = text_types.get(label, 0) + 1

        return {
            'name': self.doc.name,
            'num_pages': len(pages) if pages else 0,
            'num_texts': len(texts),
            'num_tables': len(tables),
            'num_pictures': len(pictures),
            'text_types': text_types
        }

    def export_full_structure(self) -> Dict[str, Any]:
        """
        Export complete document structure.

        Returns:
            Dictionary containing all structure information
        """
        return {
            'summary': self.get_document_summary(),
            'hierarchy': self.get_document_hierarchy(),
            'tables': self.get_tables_info(),
            'pictures': self.get_pictures_info()
        }
    
visual = DocumentStructureVisualizer(analyzer.doc)

## Tabular normalization


In [ ]:
docling_document = analyzer.doc
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple
import pandas as pd
import regex as re

# =========================
# Unit + Multiplier Models
# =========================

@dataclass
class UnitContext:
    unit: str
    multiplier: float
    source: str


@dataclass
class TableInfo:
    table_number: int
    page: int
    caption: str
    table_unit: str
    table_multiplier: float
    table_multiplier_source: str
    column_overrides: Dict[int, dict]
    original_df: pd.DataFrame
    normalized_df: pd.DataFrame
    audit_df: pd.DataFrame
    shape: tuple
    is_empty: bool

@dataclass
class AuditRow:
    table: int
    page: Optional[int]
    row: int
    column: str
    raw_text: str
    raw_value: Optional[float]
    adjusted_value: Optional[float]
    unit: Optional[str]
    multiplier: Optional[float]
    source: Optional[str]


# =========================
# Multiplier Detection
# =========================

class MultiplierDetector:
    PATTERNS = {
        r"million|millions|\$ *millions": 1_000_000,
        r"thousand|thousands|\$ *thousands": 1_000,
        r"billion|billions|\$ *billions": 1_000_000_000,
    }

    @staticmethod
    def detect_multiplier(text: str) -> Tuple[float, str]:
        if not text or not isinstance(text, str):
            return 1.0, ""

        lowered = text.lower()
        for pattern, mult in MultiplierDetector.PATTERNS.items():
            if re.search(pattern, lowered):
                return mult, text

        return 1.0, ""


# =========================
# Numeric Parsing
# =========================

def _parse_numeric_cell(cell: Any) -> Optional[float]:
    if cell is None:
        return None

    text = str(cell).strip()

    # Handle ($123.4) negatives
    neg = False
    if text.startswith("(") and text.endswith(")"):
        neg = True
        text = text[1:-1]

    text = text.replace("$", "").replace(",", "").strip()

    if re.fullmatch(r"-?\d+(\.\d+)?", text):
        val = float(text)
        return -val if neg else val

    return None


# =========================
# Unit Detection
# =========================

def detect_table_unit_context(
    df: pd.DataFrame,
    caption: Optional[str]
) -> UnitContext:
    texts = []

    if caption:
        texts.append(caption)

    texts.extend([str(c) for c in df.columns if isinstance(c, str)])

    if not df.empty:
        texts.extend(df.iloc[0].astype(str).tolist())

    for text in texts:
        mult, source = MultiplierDetector.detect_multiplier(text)
        if mult > 1:
            return UnitContext(
                unit="USD",
                multiplier=mult,
                source=f"table context: {source}"
            )

    return UnitContext(
        unit="UNKNOWN",
        multiplier=1.0,
        source="default"
    )


def detect_column_unit_overrides(
    df: pd.DataFrame,
    table_ctx: UnitContext
) -> Dict[int, UnitContext]:
    overrides = {}

    for col_idx, col_name in enumerate(df.columns):
        if not isinstance(col_name, str):
            continue

        mult, source = MultiplierDetector.detect_multiplier(col_name)

        # Only override if different from table context
        if mult > 1 and mult != table_ctx.multiplier:
            overrides[col_idx] = UnitContext(
                unit="USD",
                multiplier=mult,
                source=f"column header: {col_name}"
            )

    return overrides


# =========================
# Main Extraction Function
# =========================

def get_tables_info(docling_document) -> List[Dict[str, Any]]:
    tables_info = []

    if not hasattr(docling_document, "tables") or not docling_document.tables:
        return tables_info

    for table_idx, table in enumerate(docling_document.tables, 1):
        try:
            df = table.export_to_dataframe(doc=docling_document)

            prov = getattr(table, "prov", [])
            page_no = prov[0].page_no if prov else None

            caption_text = getattr(table, "caption_text", None)
            caption = caption_text if caption_text and not callable(caption_text) else None

            # ---- Unit detection ----
            table_ctx = detect_table_unit_context(df, caption)
            column_overrides = detect_column_unit_overrides(df, table_ctx)

            normalized_df = df.copy()
            audit_rows = []

            for row_idx, row in df.iterrows():
                for col_idx, col_name in enumerate(df.columns):
                    raw_text = str(row[col_name])
                    raw_value = _parse_numeric_cell(raw_text)

                    # Skip label column
                    if col_idx == 0:
                        audit_rows.append({
                            "table": table_idx,
                            "page": page_no,
                            "row": row_idx,
                            "column": col_name,
                            "raw_text": raw_text,
                            "raw_value": None,
                            "adjusted_value": None,
                            "unit": None,
                            "multiplier": None,
                            "source": "label"
                        })
                        continue

                    ctx = column_overrides.get(col_idx, table_ctx)

                    if raw_value is not None:
                        adjusted_value = raw_value * ctx.multiplier
                        normalized_df.at[row_idx, col_name] = adjusted_value
                    else:
                        adjusted_value = None

                    audit_rows.append(AuditRow(
                        table=table_idx,
                        page=page_no,
                        row=row_idx,
                        column=col_name,
                        raw_text=raw_text,
                        raw_value=raw_value,
                        adjusted_value=adjusted_value,
                        unit=ctx.unit,
                        multiplier=ctx.multiplier,
                        source=ctx.source
                    ))


            audit_df = pd.DataFrame(audit_rows)

            tables_info.append(TableInfo(
                table_number=table_idx,
                page=page_no,
                caption=caption,
                table_unit=table_ctx.unit,
                table_multiplier=table_ctx.multiplier,
                table_multiplier_source=table_ctx.source,
                column_overrides={idx: vars(ctx) for idx, ctx in column_overrides.items()},
                original_df=df,
                normalized_df=normalized_df,
                audit_df=audit_df,
                shape=df.shape,
                is_empty=df.empty
            ))


        except Exception as e:
            print(f"Warning: Could not process table {table_idx}: {e}")

    return tables_info

table_i = get_tables_info(docling_document)
for table in table_i:
    print(f"Table {table.table_number} | Page: {table.page} | Caption: {table.caption}")
    print("Normalized DF head:")
    print(table.normalized_df.head())
    print("\nAudit rows sample:")
    print(table.audit_df.head())

In [50]:
import re
import json
from dataclasses import dataclass, asdict
from typing import Any, List, Tuple, Optional, Dict
from pathlib import Path
from docling.document_converter import DocumentConverter
from docling_core.types.doc import DocItemLabel, TableCell
import pandas as pd

@dataclass
class NumberCandidate:
    raw_value: float
    adjusted_value: float
    text: str
    context: str
    page: int
    position: dict
    multiplier: float = 1.0
    multiplier_source: str = ""
    confidence: float = 1.0
    element_type: str = ""
    table_context: str = ""
    
    def to_dict(self):
        return asdict(self)

@dataclass
class UnitContext:
    unit: str
    multiplier: float
    source: str

@dataclass
class AuditRow:
    table: int
    page: Optional[int]
    row: int
    column: str
    raw_text: str
    raw_value: Optional[float]
    adjusted_value: Optional[float]
    unit: Optional[str]
    multiplier: Optional[float]
    source: Optional[str]

@dataclass
class TableInfo:
    table_number: int
    page: Optional[int]
    caption: Optional[str]
    table_unit: str
    table_multiplier: float
    table_multiplier_source: str
    column_overrides: Dict[int, dict]
    original_df: pd.DataFrame
    normalized_df: pd.DataFrame
    audit_df: pd.DataFrame
    shape: tuple
    is_empty: bool

# -------------------- Multiplier Detector --------------------
class MultiplierDetector:
    SCALE_WORDS = {
        'thousand': 1_000, 'thousands': 1_000,
        'million': 1_000_000, 'millions': 1_000_000,
        'billion': 1_000_000_000, 'billions': 1_000_000_000,
        'trillion': 1_000_000_000_000, 'trillions': 1_000_000_000_000,
    }
    SUFFIX_MAP = {'K': 1_000, 'M': 1_000_000, 'B': 1_000_000_000, 'T': 1_000_000_000_000, 'MM': 1_000_000}

    @classmethod
    def detect_multiplier(cls, text: str, extended_context: str = "") -> Tuple[float, str]:
        search_text = (text + " " + extended_context).lower()
        # Strategy 1: Natural language words
        for word, mult in cls.SCALE_WORDS.items():
            pattern = rf'\b(?:in\s+)?(?:\()?{word}\)?'
            if re.search(pattern, search_text):
                return mult, word
        # Strategy 2: 000s patterns
        if re.search(r"\b\d*000[\'']?s?\b", search_text):
            return 1_000, "000s"
        # Strategy 3: Currency suffix
        for suffix, mult in cls.SUFFIX_MAP.items():
            if re.search(rf'\${suffix}\b', search_text, re.I):
                return mult, f"${suffix}"
        return 1.0, ""

# -------------------- Number Extractor --------------------
class NumberExtractor:
    NUMBER_PATTERN = re.compile(
        r'''
        (?:[\$€£¥]?\s*)?            # Optional currency
        [\(]?                       # Optional opening parenthesis
        (\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d+(?:\.\d+)?)  # Number
        [\)]?                       # Optional closing parenthesis
        (?:                         # Optional suffix
            (?<=\d)                 # Must directly follow digit
            (K|M|B|T|MM)            # Valid suffix
            \b                      # Word boundary
        )?
        ''',
        re.VERBOSE | re.IGNORECASE
    )

    @classmethod
    def extract_numbers(cls, text: str) -> List[Tuple[float, str, int, int, bool]]:
        results = []
        for match in cls.NUMBER_PATTERN.finditer(text):
            matched_text = match.group(0)
            start, end = match.span()
            suffix = match.group(2)
            has_suffix = False
            if suffix and suffix.upper() in {"K", "M", "B", "T", "MM"} and suffix.isupper():
                has_suffix = True
            try:
                value = cls.parse_number(matched_text)
                if value is not None:
                    results.append((value, matched_text, start, end, has_suffix))
            except ValueError:
                continue
        return results

    @classmethod
    def parse_number(cls, text: str) -> Optional[float]:
        text = text.strip()
        is_negative = text.startswith('(') and text.endswith(')')
        # Remove currency and parentheses
        clean = re.sub(r'[\$€£¥,\(\)]', '', text)
        # Reject lone letters separated by space ("329 t")
        clean = re.sub(r'\s+[kmbt]\b', '', clean, flags=re.I)
        multiplier = 1.0
        clean_upper = clean.upper()
        if clean_upper.endswith('MM'):
            multiplier = 1_000_000
            clean = clean[:-2]
        elif clean_upper.endswith('M'):
            multiplier = 1_000_000
            clean = clean[:-1]
        elif clean_upper.endswith('B'):
            multiplier = 1_000_000_000
            clean = clean[:-1]
        elif clean_upper.endswith('K'):
            multiplier = 1_000
            clean = clean[:-1]
        elif clean_upper.endswith('T'):
            multiplier = 1_000_000_000_000
            clean = clean[:-1]
        try:
            value = float(clean.strip()) * multiplier
            return -value if is_negative else value
        except ValueError:
            return None

# -------------------- PDF Analyzer --------------------
class DoclingPDFAnalyzer:     
    def __init__(self, pdf_path: str):
        self.pdf_path = Path(pdf_path)
        self.candidates: List[NumberCandidate] = []
        self.doc_converter = DocumentConverter()
        self.debug = False
        self.doc = None  

    def _detect_table_unit_context(self, df: pd.DataFrame, caption: Optional[str]) -> UnitContext:
        texts = [caption] if caption else []
        texts.extend([str(c) for c in df.columns if isinstance(c, str)])
        if not df.empty:
            texts.extend(df.iloc[0].astype(str).tolist())
        for text in texts:
            mult, source = MultiplierDetector.detect_multiplier(text)
            if mult > 1:
                return UnitContext(unit="USD", multiplier=mult, source=f"table context: {source}")
        return UnitContext(unit="UNKNOWN", multiplier=1.0, source="default")

    def _detect_column_unit_overrides(self, df: pd.DataFrame, table_ctx: UnitContext) -> Dict[int, UnitContext]:
        overrides = {}
        for col_idx, col_name in enumerate(df.columns):
            if not isinstance(col_name, str):
                continue
            mult, source = MultiplierDetector.detect_multiplier(col_name)
            if mult > 1 and mult != table_ctx.multiplier:
                overrides[col_idx] = UnitContext(unit="USD", multiplier=mult, source=f"column header: {col_name}")
        return overrides

    def _extract_from_text_element(self, item, text, element_type, page, level):
        element_mult, element_mult_source = MultiplierDetector.detect_multiplier(text)
        numbers = NumberExtractor.extract_numbers(text)
        for value, matched_text, start, end, has_suffix in numbers:
            context_start = max(0, start - 100)
            context_end = min(len(text), end + 100)
            context = text[context_start:context_end]
            if has_suffix:
                local_mult = 1.0
                local_source = "built-in suffix"
            else:
                local_mult, local_source = MultiplierDetector.detect_multiplier(context, text[:200])
                if local_mult == 1.0 and element_mult > 1.0:
                    local_mult = element_mult
                    local_source = element_mult_source
            adjusted = abs(value * local_mult)
            candidate = NumberCandidate(
                raw_value=value,
                adjusted_value=adjusted,
                text=matched_text,
                context=context.strip(),
                page=page,
                position={'start': start, 'end': end, 'level': level},
                multiplier=local_mult,
                multiplier_source=local_source,
                confidence=self._calculate_confidence(value, local_mult, context, element_type, has_suffix),
                element_type=element_type
            )
            self.candidates.append(candidate)

    def _calculate_confidence(self, value: float, multiplier: float, context: str, element_type: str, has_suffix: bool = False) -> float:
        confidence = 1.0
        if has_suffix:
            confidence *= 1.2
        if element_type == "Table":
            confidence *= 1.15
        if value < 10 and multiplier > 1000 and not has_suffix:
            confidence *= 0.7
        if '%' in context or 'percent' in context.lower():
            confidence *= 0.5
        if 'per share' in context.lower() or 'per unit' in context.lower():
            confidence *= 0.6
        if 'footnote' in element_type.lower():
            confidence *= 0.8
        return min(confidence, 1.0)

    def _extract_with_semantic_chunking(self, doc):
        for item, level in doc.iterate_items():
            label = getattr(item, "label", None)
            text = getattr(item, "text", "")
            if not text:
                continue
            page = getattr(item, "page", 1)
            if label != DocItemLabel.TABLE:
                self._extract_from_text_element(item, text, str(label) if label else "Unknown", page, level)
        tables_info = self.get_tables_info()
        self._add_table_candidates(tables_info)

    def _add_table_candidates(self, tables_info: List[TableInfo]):
        for table in tables_info:
            for _, row in table.audit_df.iterrows():
                if row["raw_value"] is None:
                    continue
                candidate = NumberCandidate(
                    raw_value=row["raw_value"],
                    adjusted_value=abs(row["adjusted_value"]) if row["adjusted_value"] is not None else abs(row["raw_value"]),
                    text=row["raw_text"],
                    context=f"Table {table.table_number}, column '{row['column']}'",
                    page=row["page"],
                    position={"table": table.table_number, "row": row["row"], "column": row["column"]},
                    multiplier=row["multiplier"] or 1.0,
                    multiplier_source=row["source"] or "table",
                    confidence=0.9,
                    element_type="TABLE"
                )
                self.candidates.append(candidate)

    def extract_from_pdf(self) -> List[NumberCandidate]:
        result = self.doc_converter.convert(str(self.pdf_path))
        self.doc = result.document 
        self._extract_with_semantic_chunking(self.doc)
        return self.candidates

    def get_tables_info(self) -> List[TableInfo]:
        tables_info: List[TableInfo] = []
        docling_document = self.doc
        if not hasattr(docling_document, "tables") or not docling_document.tables:
            return tables_info
        for table_idx, table in enumerate(docling_document.tables, 1):
            try:
                df = table.export_to_dataframe(doc=docling_document)
                prov = getattr(table, "prov", [])
                page_no = prov[0].page_no if prov else None
                caption_text = getattr(table, "caption_text", None)
                caption = caption_text if caption_text and not callable(caption_text) else None
                table_ctx = self._detect_table_unit_context(df, caption)
                column_overrides = self._detect_column_unit_overrides(df, table_ctx)
                normalized_df = df.copy()
                audit_rows: List[AuditRow] = []
                for row_idx, row in df.iterrows():
                    for col_idx, col_name in enumerate(df.columns):
                        raw_text = str(row[col_name])
                        raw_value = NumberExtractor.parse_number(raw_text)
                        if col_idx == 0:
                            audit_rows.append(AuditRow(
                                table=table_idx,
                                page=page_no,
                                row=row_idx,
                                column=col_name,
                                raw_text=raw_text,
                                raw_value=None,
                                adjusted_value=None,
                                unit=None,
                                multiplier=None,
                                source="label"
                            ))
                            continue
                        ctx = column_overrides.get(col_idx, table_ctx)
                        adjusted_value = raw_value * ctx.multiplier if raw_value is not None else None
                        normalized_df.at[row_idx, col_name] = adjusted_value if adjusted_value is not None else raw_text
                        audit_rows.append(AuditRow(
                            table=table_idx,
                            page=page_no,
                            row=row_idx,
                            column=col_name,
                            raw_text=raw_text,
                            raw_value=raw_value,
                            adjusted_value=adjusted_value,
                            unit=ctx.unit,
                            multiplier=ctx.multiplier,
                            source=ctx.source
                        ))
                audit_df = pd.DataFrame([asdict(row) for row in audit_rows])
                tables_info.append(TableInfo(
                    table_number=table_idx,
                    page=page_no,
                    caption=caption,
                    table_unit=table_ctx.unit,
                    table_multiplier=table_ctx.multiplier,
                    table_multiplier_source=table_ctx.source,
                    column_overrides={idx: asdict(ctx) for idx, ctx in column_overrides.items()},
                    original_df=df,
                    normalized_df=normalized_df,
                    audit_df=audit_df,
                    shape=df.shape,
                    is_empty=df.empty
                ))
            except Exception as e:
                print(f"Warning: Could not process table {table_idx}: {e}")
        return tables_info

    def rank_candidates(self, top_n: int = 20) -> List[NumberCandidate]:
        valid_candidates = [c for c in self.candidates if c.adjusted_value < 1e15]
        ranked = sorted(valid_candidates, key=lambda x: (x.adjusted_value, x.confidence), reverse=True)
        return ranked[:top_n]

    def generate_report(self, top_n: int = 20) -> str:
        top_candidates = self.rank_candidates(top_n)
        report = [f"# PDF Number Analysis Report\n**Document:** {self.pdf_path.name}\n**Total numbers found:** {len(self.candidates)}\n\n## Top Numbers\n"]
        if not self.candidates:
            report.append("No numbers found.")
            return "\n".join(report)
        largest_raw = max(self.candidates, key=lambda x: abs(x.raw_value))
        largest_adjusted = top_candidates[0]
        report.append(f"### Largest Raw Number: {largest_raw.raw_value:,.2f}\n- Page: {largest_raw.page}\n- Type: {largest_raw.element_type}\n- Text: `{largest_raw.text}`\n")
        report.append(f"### Largest Adjusted Number: {largest_adjusted.adjusted_value:,.2f}\n- Raw: {largest_adjusted.raw_value:,.2f}\n- Multiplier: {largest_adjusted.multiplier} ({largest_adjusted.multiplier_source})\n- Page: {largest_adjusted.page}\n- Type: {largest_adjusted.element_type}\n")
        report.append("\n### Top Candidates\n| Rank | Adjusted | Raw | Multiplier | Page | Type | Context |\n")
        for i, c in enumerate(top_candidates, 1):
            ctx = c.context[:80].replace('\n', ' ').replace('|', '/')
            report.append(f"| {i} | {c.adjusted_value:,.0f} | {c.raw_value:,.2f} | {c.multiplier:,.0f}x | {c.page} | {c.element_type} | {ctx}... |")
        return "\n".join(report)

    def save_results(self, output_dir: str = "results"):
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        top_candidates = self.rank_candidates(50)
        with open(output_path / "top_candidates.json", 'w') as f:
            json.dump([c.to_dict() for c in top_candidates], f, indent=2)
        report = self.generate_report()
        with open(output_path / "analysis_report.md", 'w') as f:
            f.write(report)
        print(f"✅ Results saved to {output_dir}/")

# =========================
# Example usage
# =========================

# Assuming docling_document is already loaded

analyzer = DoclingPDFAnalyzer("FY25_Air_Force.pdf")
analyzer.extract_from_pdf()   # this will populate analyzer.doc
tables_info = analyzer.get_tables_info()  # no arguments needed

from dataclasses import asdict

for table in tables_info:
    print("TableInfo:")
    for k, v in asdict(table).items():
        if isinstance(v, pd.DataFrame):
            print(f"  {k}: DataFrame shape={v.shape}")
        else:
            print(f"  {k}: {v}")
    
    print("\nAudit rows sample:")
    print(table.audit_df.head())

    print("=" * 60)

TableInfo:
  table_number: 1
  page: 13
  caption: None
  table_unit: USD
  table_multiplier: 1000000
  table_multiplier_source: table context: million
  column_overrides: {}
  original_df: DataFrame shape=(11, 4)
  normalized_df: DataFrame shape=(11, 4)
  audit_df: DataFrame shape=(44, 10)
  shape: (11, 4)
  is_empty: False

Audit rows sample:
   table  page  row                 column            raw_text  raw_value  \
0      1    13    0  (Dollars in Millions)       Total Revenue        NaN   
1      1    13    0                FY 2023            28,239.2    28239.2   
2      1    13    0                FY 2024            29,176.6    29176.6   
3      1    13    0                FY 2025            30,704.1    30704.1   
4      1    13    1  (Dollars in Millions)  Cost of Goods Sold        NaN   

   adjusted_value  unit  multiplier                  source  
0             NaN  None         NaN                   label  
1    2.823920e+10   USD   1000000.0  table context: million  
2   

In [ ]:
import re
import json
from dataclasses import dataclass, asdict
from typing import Any, List, Tuple, Optional, Dict
from pathlib import Path
from docling.document_converter import DocumentConverter
from docling_core.types.doc import DocItemLabel, TableCell
import pandas as pd

@dataclass
class NumberCandidate:
    """Represents a number found in the document with its context."""
    raw_value: float
    adjusted_value: float
    text: str
    context: str
    page: int
    position: dict
    multiplier: float = 1.0
    multiplier_source: str = ""
    confidence: float = 1.0
    element_type: str = ""
    table_context: str = ""
    
    def to_dict(self):
        return asdict(self)

@dataclass
class UnitContext:
    unit: str
    multiplier: float
    source: str

@dataclass
class AuditRow:
    table: int
    page: Optional[int]
    row: int
    column: str
    raw_text: str
    raw_value: Optional[float]
    adjusted_value: Optional[float]
    unit: Optional[str]
    multiplier: Optional[float]
    source: Optional[str]

@dataclass
class TableInfo:
    table_number: int
    page: Optional[int]
    caption: Optional[str]
    table_unit: str
    table_multiplier: float
    table_multiplier_source: str
    column_overrides: Dict[int, dict]
    original_df: pd.DataFrame
    normalized_df: pd.DataFrame
    audit_df: pd.DataFrame
    shape: tuple
    is_empty: bool


class MultiplierDetector:
    """Detects scale multipliers using multiple strategies."""
    SCALE_WORDS = {
        'thousand': 1_000, 'thousands': 1_000,
        'million': 1_000_000, 'millions': 1_000_000,
        'billion': 1_000_000_000, 'billions': 1_000_000_000,
        'trillion': 1_000_000_000_000, 'trillions': 1_000_000_000_000,
    }
    
    SUFFIX_MAP = {'k': 1_000, 'm': 1_000_000, 'b': 1_000_000_000, 't': 1_000_000_000_000, 'mm': 1_000_000}
    
    @classmethod
    def detect_multiplier(cls, text: str, extended_context: str = "") -> Tuple[float, str]:
        """Detect multiplier using hierarchical pattern matching."""
        search_text = (text + " " + extended_context).lower()
        
        # Strategy 1: Direct word matching (most reliable)
        for word, mult in cls.SCALE_WORDS.items():
            # Match variations: "in millions", "(millions)", "millions of", or just "millions" nearby
            pattern = rf'\b(?:in\s+)?(?:\()?{word}\)?'
            if re.search(pattern, search_text):
                return mult, word
        
        # Strategy 2: Numeric patterns like "000s" or "000's"
        if re.search(r"\b000[\'']?s?\b", search_text):
            return 1_000, "000s"
        
        # Strategy 3: Currency suffixes like "$M"
        for suffix, mult in cls.SUFFIX_MAP.items():
            if re.search(rf'\${suffix}\b', search_text, re.I):
                return mult, f"${suffix}"
        
        return 1.0, ""

class NumberExtractor:
    """Extracts numbers from text with various format support."""
    
    # Comprehensive number pattern
    NUMBER_PATTERN = re.compile(
        r'(?:[\$€£¥]?\s*)?'  # Optional currency symbol
        r'[\(]?'  # Optional opening parenthesis (negative)
        r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d+(?:\.\d+)?)'  # Number - supports unlimited digits
        r'[\)]?'  # Optional closing parenthesis
        r'(?:\s*([MBKTmm]))?'  # Optional M, B, K, T suffix - CAPTURED
    , re.IGNORECASE)
    
    @classmethod
    def extract_numbers(cls, text: str) -> List[Tuple[float, str, int, int, bool]]:
        """
        Extract all numbers from text.
        Returns list of (value, matched_text, start_pos, end_pos, has_suffix)
        """
        results = []
        
        for match in cls.NUMBER_PATTERN.finditer(text):
            matched_text = match.group(0)
            start, end = match.span()
            suffix = match.group(2)  # Get the M/B/K/T suffix if present
            
            try:
                value = cls.parse_number(matched_text)
                if value is not None:
                    has_suffix = suffix is not None
                    results.append((value, matched_text, start, end, has_suffix))
            except ValueError:
                continue
        
        return results
    
    @classmethod
    def parse_number(cls, text: str) -> Optional[float]:
        """Parse a number string into float, handling various formats."""
        text = text.strip()
        
        # Handle parenthetical negatives
        is_negative = text.startswith('(') and text.endswith(')')
        
        # Remove currency symbols, parentheses, commas
        clean = re.sub(r'[\$€£¥,\(\)]', '', text)
        
        # Handle M, B, K, T, MM suffixes (case insensitive)
        multiplier = 1.0
        clean_upper = clean.upper()
        if clean_upper.endswith('MM'):
            multiplier = 1_000_000
            clean = clean[:-2]
        elif clean_upper.endswith('M'):
            multiplier = 1_000_000
            clean = clean[:-1]
        elif clean_upper.endswith('B'):
            multiplier = 1_000_000_000
            clean = clean[:-1]
        elif clean_upper.endswith('K'):
            multiplier = 1_000
            clean = clean[:-1]
        elif clean_upper.endswith('T'):
            multiplier = 1_000_000_000_000
            clean = clean[:-1]
        
        try:
            value = float(clean.strip()) * multiplier
            return -value if is_negative else value
        except ValueError:
            return None
        



class DoclingPDFAnalyzer:     
    def __init__(self, pdf_path: str):
        self.pdf_path = Path(pdf_path)
        self.candidates: List[NumberCandidate] = []
        self.doc_converter = DocumentConverter()
        self.debug = False
        self.doc = None  
    
    def _detect_table_unit_context(
        self, 
        df: pd.DataFrame,
        caption: Optional[str]
    ) -> UnitContext:
        texts = []

        if caption:
            texts.append(caption)

        texts.extend([str(c) for c in df.columns if isinstance(c, str)])

        if not df.empty:
            texts.extend(df.iloc[0].astype(str).tolist())

        for text in texts:
            mult, source = MultiplierDetector.detect_multiplier(text)
            if mult > 1:
                return UnitContext(
                    unit="USD",
                    multiplier=mult,
                    source=f"table context: {source}"
                )

        return UnitContext(
            unit="UNKNOWN",
            multiplier=1.0,
            source="default"
        )


    def _detect_column_unit_overrides(
        self, 
        df: pd.DataFrame,
        table_ctx: UnitContext
    ) -> Dict[int, UnitContext]:
        overrides = {}

        for col_idx, col_name in enumerate(df.columns):
            if not isinstance(col_name, str):
                continue

            mult, source = MultiplierDetector.detect_multiplier(col_name)

            # Only override if different from table context
            if mult > 1 and mult != table_ctx.multiplier:
                overrides[col_idx] = UnitContext(
                    unit="USD",
                    multiplier=mult,
                    source=f"column header: {col_name}"
                )

        return overrides
    


    def _extract_from_table(self):
        """Extract numbers from tables using declarative multiplier detection."""
        table_i = self.get_tables_info()

        from dataclasses import asdict

        for table in table_i:
            print("TableInfo:")
            for k, v in asdict(table).items():
                if isinstance(v, pd.DataFrame):
                    print(f"  {k}: DataFrame shape={v.shape}")
                else:
                    print(f"  {k}: {v}")
            
            print("\nAudit rows sample:")
            print(table.audit_df.head())

            print("=" * 60)

        return 
        

    def _extract_with_semantic_chunking(self, doc):
        print("🔍 Extracting with semantic chunking...")

        table_count = 0
        text_count = 0

        for item, level in doc.iterate_items():
            label = getattr(item, "label", None)
            text = getattr(item, "text", "")

            if not text:
                continue

            page = getattr(item, "page", 1)

            if label == DocItemLabel.TABLE:
                table_count += 1
                continue
            else:
                text_count += 1
                self._extract_from_text_element(
                    item,
                    text,
                    str(label) if label else "Unknown",
                    page,
                    level,
                )

        tables_info = self.get_tables_info()
        self._add_table_candidates(tables_info)

        print(f"✅ Processed {table_count} tables and {text_count} text elements")

    
    def _extract_from_text_element(self, item, text, element_type, page, level):
        """Extract numbers from non-table text elements."""
        # Detect element-level multiplier
        element_mult, element_mult_source = MultiplierDetector.detect_multiplier(text)
        
        # Extract numbers
        numbers = NumberExtractor.extract_numbers(text)
        
        for value, matched_text, start, end, has_suffix in numbers:
            # Get local context
            context_start = max(0, start - 100)
            context_end = min(len(text), end + 100)
            context = text[context_start:context_end]
            
            # Determine multiplier
            if has_suffix:
                local_mult = 1.0
                local_source = "built-in suffix"
            else:
                local_mult, local_source = MultiplierDetector.detect_multiplier(context, text[:200])
                
                # Use element-level multiplier if no local one
                if local_mult == 1.0 and element_mult > 1.0:
                    local_mult = element_mult
                    local_source = element_mult_source
            
            adjusted = abs(value * local_mult)
            
            if self.debug and adjusted > 1_000_000_000:
                print(f"  DEBUG: {matched_text} × {local_mult}x = {adjusted:,.0f} (from {local_source})")
            
            candidate = NumberCandidate(
                raw_value=value,
                adjusted_value=adjusted,
                text=matched_text,
                context=context.strip(),
                page=page,
                position={'start': start, 'end': end, 'level': level},
                multiplier=local_mult,
                multiplier_source=local_source,
                confidence=self._calculate_confidence(value, local_mult, context, element_type, has_suffix),
                element_type=element_type
            )
            
            self.candidates.append(candidate)
    

    def _calculate_confidence(self, value: float, multiplier: float, context: str, element_type: str, has_suffix: bool = False) -> float:
        """Calculate confidence score for a number."""
        confidence = 1.0
        
        # Higher confidence if number has explicit suffix (already scaled correctly)
        if has_suffix:
            confidence *= 1.2
        
        # Higher confidence for table numbers (structured data)
        if element_type == "Table":
            confidence *= 1.15
        
        # Lower confidence for very small numbers that got huge multipliers
        if value < 10 and multiplier > 1000 and not has_suffix:
            confidence *= 0.7
        
        # Check if context suggests this might be a percentage or ratio
        if '%' in context or 'percent' in context.lower():
            confidence *= 0.5
        
        # Check for "per share" or similar that might indicate smaller values
        if 'per share' in context.lower() or 'per unit' in context.lower():
            confidence *= 0.6
        
        # Lower confidence for numbers in footnotes
        if 'footnote' in element_type.lower():
            confidence *= 0.8
        
        return min(confidence, 1.0)  # Cap at 1.0
    

    def _add_table_candidates(self, tables_info: List[TableInfo]):
        """Convert table audit rows into NumberCandidate objects."""
        
        for table in tables_info:
            for _, row in table.audit_df.iterrows():

                if row["raw_value"] is None:
                    continue

                candidate = NumberCandidate(
                    raw_value=row["raw_value"],
                    adjusted_value=abs(row["adjusted_value"]) if row["adjusted_value"] is not None else abs(row["raw_value"]),
                    text=row["raw_text"],
                    context=f"Table {table.table_number}, column '{row['column']}'",
                    page=row["page"],
                    position={
                        "table": table.table_number,
                        "row": row["row"],
                        "column": row["column"],
                    },
                    multiplier=row["multiplier"] or 1.0,
                    multiplier_source=row["source"] or "table",
                    confidence=0.9,
                    element_type="TABLE"
                )

                self.candidates.append(candidate)

    def extract_from_pdf(self) -> List[NumberCandidate]:
        print(f"📄 Processing with Docling: {self.pdf_path.name}")

        result = self.doc_converter.convert(str(self.pdf_path))
        self.doc = result.document 

        print("✅ Docling conversion complete")

        self._extract_with_semantic_chunking(self.doc)
        return self.candidates


    def get_tables_info(self) -> List[TableInfo]:
        tables_info: List[TableInfo] = []
        docling_document = self.doc

        if not hasattr(docling_document, "tables") or not docling_document.tables:
            return tables_info

        for table_idx, table in enumerate(docling_document.tables, 1):
            try:
                # Extract DataFrame
                df = table.export_to_dataframe(doc=docling_document)

                # Page info
                prov = getattr(table, "prov", [])
                page_no = prov[0].page_no if prov else None

                # Caption
                caption_text = getattr(table, "caption_text", None)
                caption = caption_text if caption_text and not callable(caption_text) else None

                # Units
                table_ctx = self._detect_table_unit_context(df, caption)
                column_overrides = self._detect_column_unit_overrides(df, table_ctx)
                normalized_df = df.copy()

                # Build AuditRows
                audit_rows: List[AuditRow] = []

                for row_idx, row in df.iterrows():
                    for col_idx, col_name in enumerate(df.columns):
                        raw_text = str(row[col_name])
                        raw_value = NumberExtractor.parse_number(raw_text)

                        # Skip label column
                        if col_idx == 0:
                            audit_rows.append(AuditRow(
                                table=table_idx,
                                page=page_no,
                                row=row_idx,
                                column=col_name,
                                raw_text=raw_text,
                                raw_value=None,
                                adjusted_value=None,
                                unit=None,
                                multiplier=None,
                                source="label"
                            ))
                            continue

                        ctx = column_overrides.get(col_idx, table_ctx)
                        adjusted_value = raw_value * ctx.multiplier if raw_value is not None else None
                        normalized_df.at[row_idx, col_name] = adjusted_value if adjusted_value is not None else raw_text

                        audit_rows.append(AuditRow(
                            table=table_idx,
                            page=page_no,
                            row=row_idx,
                            column=col_name,
                            raw_text=raw_text,
                            raw_value=raw_value,
                            adjusted_value=adjusted_value,
                            unit=ctx.unit,
                            multiplier=ctx.multiplier,
                            source=ctx.source
                        ))

                # Convert AuditRows to DataFrame safely
                audit_df = pd.DataFrame([asdict(row) for row in audit_rows])

                # Append table info
                tables_info.append(TableInfo(
                    table_number=table_idx,
                    page=page_no,
                    caption=caption,
                    table_unit=table_ctx.unit,
                    table_multiplier=table_ctx.multiplier,
                    table_multiplier_source=table_ctx.source,
                    column_overrides={idx: asdict(ctx) for idx, ctx in column_overrides.items()},
                    original_df=df,
                    normalized_df=normalized_df,
                    audit_df=audit_df,
                    shape=df.shape,
                    is_empty=df.empty
                ))

            except Exception as e:
                print(f"Warning: Could not process table {table_idx}: {e}")

        return tables_info

    def rank_candidates(self, top_n: int = 20) -> List[NumberCandidate]:
        """Rank candidates by adjusted value."""
        # Filter out obvious outliers
        valid_candidates = [
            c for c in self.candidates 
            if c.adjusted_value < 1e15  # Reasonable max value
        ]
        
        # Sort by adjusted value, then by confidence as tiebreaker
        ranked = sorted(valid_candidates, key=lambda x: (x.adjusted_value, x.confidence), reverse=True)
        return ranked[:top_n]
    
    
    def generate_report(self, top_n: int = 20) -> str:
        """Generate a markdown report of findings."""
        top_candidates = self.rank_candidates(top_n)
        
        report = [
            "# PDF Number Analysis Report (Docling Semantic)\n",
            f"**Document:** {self.pdf_path.name}\n",
            f"**Total numbers found:** {len(self.candidates)}\n",
            f"**Extraction method:** Docling Semantic Chunking with Native Table Understanding\n",
            "\n## Top Numbers Found\n",
        ]
        
        # Largest raw number
        largest_raw = max(self.candidates, key=lambda x: abs(x.raw_value))
        report.append(f"\n### Largest Raw Number: {largest_raw.raw_value:,.2f}")
        report.append(f"- **Page:** {largest_raw.page}")
        report.append(f"- **Element Type:** {largest_raw.element_type}")
        report.append(f"- **Text:** `{largest_raw.text}`")
        report.append(f"- **Context:** {largest_raw.context[:200]}...\n")
        
        # Largest adjusted number
        largest_adjusted = top_candidates[0]
        report.append(f"\n### Largest Adjusted Number: {largest_adjusted.adjusted_value:,.2f}")
        report.append(f"- **Raw value:** {largest_adjusted.raw_value:,.2f}")
        report.append(f"- **Multiplier:** {largest_adjusted.multiplier:,.0f}x ({largest_adjusted.multiplier_source})")
        report.append(f"- **Page:** {largest_adjusted.page}")
        report.append(f"- **Element Type:** {largest_adjusted.element_type}")
        report.append(f"- **Confidence:** {largest_adjusted.confidence:.1%}")
        report.append(f"- **Context:** {largest_adjusted.context[:200]}...")
        if largest_adjusted.table_context:
            report.append(f"- **Table Context:** {largest_adjusted.table_context[:100]}...\n")
        
        # Top candidates table
        report.append("\n### Top Candidates\n")
        report.append("| Rank | Adjusted Value | Raw Value | Multiplier | Page | Type | Context |")
        
        for i, candidate in enumerate(top_candidates, 1):
            context_preview = candidate.context[:80].replace('\n', ' ').replace('|', '/')
            report.append(
                f"| {i} | {candidate.adjusted_value:,.0f} | "
                f"{candidate.raw_value:,.2f} | {candidate.multiplier:,.0f}x | "
                f"{candidate.page} | {candidate.element_type} | {context_preview}... |"
            )
        
        return '\n'.join(report)
    
    def save_results(self, output_dir: str = "results"):
        """Save results to JSON and markdown."""
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        
        # Save top candidates as JSON
        top_candidates = self.rank_candidates(50)
        with open(output_path / "top_candidates.json", 'w') as f:
            json.dump([c.to_dict() for c in top_candidates], f, indent=2)
        
        # Save markdown report
        report = self.generate_report()
        with open(output_path / "analysis_report.md", 'w') as f:
            f.write(report)
        
        print(f"✅ Results saved to {output_dir}/")

def main(
    pdf_path: str,
    top_n: int = 20,
    output_dir: str = "results",
    debug: bool = False,
    no_chunking: bool = False,
):
    analyzer = DoclingPDFAnalyzer(pdf_path)
    analyzer.debug = debug

    if no_chunking:
        analyzer.use_chunking = False  # or whatever flag your analyzer uses

    analyzer.extract_from_pdf()

    print("\n" + analyzer.generate_report(top_n))
    analyzer.save_results(output_dir)

    if analyzer.candidates:
        top = analyzer.rank_candidates(1)[0]
        print(f"\n🎯 ANSWER:")
        print(
            f"   Largest raw number: "
            f"{max(analyzer.candidates, key=lambda x: abs(x.raw_value)).raw_value:,.2f}"
        )
        print(f"   Largest adjusted number: {top.adjusted_value:,.2f}")
        print(f"   (from {top.raw_value:,.2f} × {top.multiplier:,.0f})")
    else:
        print("\n⚠️  No numbers found in document")

    return analyzer


def cli():
    import argparse

    parser = argparse.ArgumentParser(
        description="Analyze numeric values in a PDF using Docling"
    )

    parser.add_argument(
        "pdf_path",
        help="Path to the PDF document to analyze",
    )

    parser.add_argument(
        "--top-n",
        type=int,
        default=20,
        help="Number of top numeric candidates to display (default: 20)",
    )

    parser.add_argument(
        "--output-dir",
        default="results",
        help="Directory to save extracted results (default: results)",
    )

    parser.add_argument(
        "--debug",
        action="store_true",
        help="Enable debug logging",
    )

    parser.add_argument(
        "--no-chunking",
        action="store_true",
        help="Disable Docling chunked retrieval",
    )

    args = parser.parse_args()

    main(
        pdf_path=args.pdf_path,
        top_n=args.top_n,
        output_dir=args.output_dir,
        debug=args.debug,
        no_chunking=args.no_chunking,
    )


if __name__ == "__main__":
    cli()


📄 Processing with Docling: TEST.pdf
✅ Docling conversion complete
🔍 Extracting with semantic chunking...
  DEBUG: 5 B × 1.0x = 5,000,000,000 (from built-in suffix)
  DEBUG: 5 B × 1.0x = 5,000,000,000 (from built-in suffix)
  DEBUG: $1,088.6 m × 1.0x = 1,088,600,000 (from built-in suffix)
  DEBUG: $1,730.8 m × 1.0x = 1,730,800,000 (from built-in suffix)
  DEBUG: $9.6 b × 1.0x = 9,600,000,000 (from built-in suffix)
  DEBUG: $2,239.3 m × 1.0x = 2,239,300,000 (from built-in suffix)
  DEBUG: $1,414.2 m × 1.0x = 1,414,200,000 (from built-in suffix)
  DEBUG: $1,449 m × 1.0x = 1,449,000,000 (from built-in suffix)
✅ Processed 0 tables and 69 text elements

# PDF Number Analysis Report (Docling Semantic)

**Document:** TEST.pdf

**Total numbers found:** 217

**Extraction method:** Docling Semantic Chunking with Native Table Understanding


## Top Numbers Found


### Largest Raw Number: 9,600,000,000.00
- **Page:** 1
- **Element Type:** text
- **Text:** `$9.6 b`
- **Context:** r, land, and sea tr

In [18]:
visual.get_document_hierarchy()

[{'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
  'text': 'UNITED STATES AIR FORCE WORKING CAPITAL FUND (Appropriation: 4930)',
  'page': 1,
  'level': 2},
 {'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
  'text': 'Fiscal Year (FY) 2025 Budget Estimates January 2024',
  'page': 1,
  'level': 2},
 {'type': <DocItemLabel.PAGE_HEADER: 'page_header'>,
  'text': 'TABLE OF CONTENTS',
  'page': 3,
  'level': 4},
 {'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
  'text': 'AIR FORCE WORKING CAPITAL FUND FISCAL YEAR (FY) 2025 BUDGET ESTIMATES',
  'page': 3,
  'level': 2},
 {'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
  'text': 'AIR FORCE WORKING CAPITAL FUND',
  'page': 5,
  'level': 2},
 {'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
  'text': 'SUMMARY',
  'page': 5,
  'level': 2},
 {'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
  'text': 'THIS PAGE INTENTIONALLY LEFT BLANK',
  'page': 6,
  'level': 2},
 {'type': <DocItemLabel.S

In [19]:
visual.get_tables_info()

[{'table_number': 1,
  'page': 13,
  'caption': None,
  'dataframe':                  (Dollars in Millions)              FY 2023   FY 2024  \
  0                        Total Revenue             28,239.2  29,176.6   
  1                   Cost of Goods Sold             27,950.4  29,494.7   
  2     Net Operating Result Adjustments                  5.0    (46.6)   
  3           Net Operating Result (NOR)                293.8   (364.7)   
  4   Accumulated Operating Result (AOR)  Result (AOR)1 546.0      73.1   
  5                Civilian End Strength               33,848    34,735   
  6                Military End Strength               12,472    12,705   
  7                   Civilian Workyears               32,839    32,973   
  8                   Military Workyears               12,388    12,885   
  9                       Capital Budget                281.2     312.6   
  10              Direct Appropriation 1                 80.4      80.4   
  
       FY 2025  
  0   30,704.

In [20]:
visual.get_document_summary()

{'name': 'TEST',
 'num_pages': 18,
 'num_texts': 119,
 'num_tables': 3,
 'num_pictures': 4,
 'text_types': {<DocItemLabel.SECTION_HEADER: 'section_header'>: 22,
  <DocItemLabel.TEXT: 'text'>: 69,
  <DocItemLabel.PAGE_HEADER: 'page_header'>: 1,
  <DocItemLabel.PAGE_FOOTER: 'page_footer'>: 23,
  <DocItemLabel.FOOTNOTE: 'footnote'>: 2,
  <DocItemLabel.CAPTION: 'caption'>: 2}}

In [21]:
visual.export_full_structure()

{'summary': {'name': 'TEST',
  'num_pages': 18,
  'num_texts': 119,
  'num_tables': 3,
  'num_pictures': 4,
  'text_types': {<DocItemLabel.SECTION_HEADER: 'section_header'>: 22,
   <DocItemLabel.TEXT: 'text'>: 69,
   <DocItemLabel.PAGE_HEADER: 'page_header'>: 1,
   <DocItemLabel.PAGE_FOOTER: 'page_footer'>: 23,
   <DocItemLabel.FOOTNOTE: 'footnote'>: 2,
   <DocItemLabel.CAPTION: 'caption'>: 2}},
 'hierarchy': [{'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
   'text': 'UNITED STATES AIR FORCE WORKING CAPITAL FUND (Appropriation: 4930)',
   'page': 1,
   'level': 2},
  {'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
   'text': 'Fiscal Year (FY) 2025 Budget Estimates January 2024',
   'page': 1,
   'level': 2},
  {'type': <DocItemLabel.PAGE_HEADER: 'page_header'>,
   'text': 'TABLE OF CONTENTS',
   'page': 3,
   'level': 4},
  {'type': <DocItemLabel.SECTION_HEADER: 'section_header'>,
   'text': 'AIR FORCE WORKING CAPITAL FUND FISCAL YEAR (FY) 2025 BUDGET ESTIMATES',
 